# 1. Cleaning and Wrangling Crime Data

## _Importing Libraries:_

In [37]:
import pandas as pd #For data manipulation

import numpy as np #For data computation array,linear alegbra,etc.


# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm #matplotlib color map
import matplotlib.colors as colors #to obtain colors for the color map


plt.style.use('ggplot') #for better looking graphs\

%matplotlib inline

import seaborn as sns #Another lib for better looking graphs 

import folium #Map rendering library 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values



## _Importing and Exploring the Dataset:_

In [39]:
df = pd.read_csv('NYPD_Complaint_Data_Current__Year_To_Date_.csv')
df.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,466502077,75,BROOKLYN,03/30/2020,17:30:00,03/31/2020,06:53:00,COMPLETED,NaN,NaN,...,NaN,NaN,25-44,BLACK,M,1018496,178657,40.656991,-73.876574,"(40.65699087900003, -73.87657444799999)"
1,303191835,77,BROOKLYN,03/28/2020,19:30:00,03/28/2020,20:30:00,COMPLETED,NaN,NaN,...,M,NaN,<18,UNKNOWN,F,1003606,185050,40.674583,-73.930222,"(40.67458330800008, -73.93022154099998)"
2,735488557,43,BRONX,03/29/2020,14:10:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,NaN,UNKNOWN,UNKNOWN,D,1019853,241853,40.830443,-73.871349,"(40.83044253800006, -73.871349147)"
3,315962428,40,BRONX,03/29/2020,07:10:00,03/29/2020,07:16:00,COMPLETED,NaN,NaN,...,M,NaN,25-44,WHITE HISPANIC,F,1007236,237260,40.817878,-73.916957,"(40.817877907000025, -73.91695668199996)"
4,165437868,114,QUEENS,03/27/2020,13:15:00,03/27/2020,14:00:00,COMPLETED,NaN,NaN,...,NaN,NaN,45-64,BLACK,F,1002018,213258,40.752011,-73.935872,"(40.75201086000004, -73.93587196099996)"


Exploring the dataset to get familiar with the size, the columns and get the general info of it.

In [40]:
print(df.shape)
df.info()

(108058, 35)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108058 entries, 0 to 108057
Data columns (total 35 columns):
CMPLNT_NUM           108058 non-null int64
ADDR_PCT_CD          108058 non-null int64
BORO_NM              107981 non-null object
CMPLNT_FR_DT         108058 non-null object
CMPLNT_FR_TM         108058 non-null object
CMPLNT_TO_DT         96506 non-null object
CMPLNT_TO_TM         96547 non-null object
CRM_ATPT_CPTD_CD     108058 non-null object
HADEVELOPT           5110 non-null object
HOUSING_PSA          7875 non-null float64
JURISDICTION_CODE    107989 non-null float64
JURIS_DESC           108058 non-null object
KY_CD                108058 non-null int64
LAW_CAT_CD           108058 non-null object
LOC_OF_OCCUR_DESC    90162 non-null object
OFNS_DESC            108057 non-null object
PARKS_NM             534 non-null object
PATROL_BORO          107989 non-null object
PD_CD                107989 non-null float64
PD_DESC              107989 non-null object
PREM_T

In [41]:
df.columns

Index(['CMPLNT_NUM', 'ADDR_PCT_CD', 'BORO_NM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM',
       'CMPLNT_TO_DT', 'CMPLNT_TO_TM', 'CRM_ATPT_CPTD_CD', 'HADEVELOPT',
       'HOUSING_PSA', 'JURISDICTION_CODE', 'JURIS_DESC', 'KY_CD', 'LAW_CAT_CD',
       'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PARKS_NM', 'PATROL_BORO', 'PD_CD',
       'PD_DESC', 'PREM_TYP_DESC', 'RPT_DT', 'STATION_NAME', 'SUSP_AGE_GROUP',
       'SUSP_RACE', 'SUSP_SEX', 'TRANSIT_DISTRICT', 'VIC_AGE_GROUP',
       'VIC_RACE', 'VIC_SEX', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude',
       'Longitude', 'Lat_Lon'],
      dtype='object')

In [42]:
df.describe(include=['object'])

,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,JURIS_DESC,LAW_CAT_CD,LOC_OF_OCCUR_DESC,...,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,Lat_Lon
count,107981,108058,108058,96506,96547,108058,5110,108058,108058,90162,...,107655,108058,2948,83132,83132,83132,108058,108058,108058,108058
unique,5,872,1440,573,1440,2,238,17,3,5,...,74,91,330,11,7,3,9,7,4,36589
top,BROOKLYN,01/01/2020,12:00:00,01/15/2020,12:00:00,COMPLETED,CASTLE HILL,N.Y. POLICE DEPT,MISDEMEANOR,INSIDE,...,STREET,01/15/2020,125 STREET,UNKNOWN,BLACK,M,25-44,UNKNOWN,F,"(40.75043076800005, -73.98928217599996)"
freq,31028,1505,2648,1257,1473,106381,104,96186,56762,60685,...,29592,1498,120,34043,31025,50060,39822,30935,42778,332


## _Wrangling the Data:_

Filtering the columns relevant to the scope of the project:

1. Borough = Name of the Borough where crime was committed

2. Crime Committed = Description of the crime committed

3. Location of Crime = The type of premise the crime was committed on

4. Latitiude =  The Latitude where crime was committed

5. Longitude = The Longitude were the crime was committed

In [43]:
crime_df = df[['BORO_NM','OFNS_DESC','PREM_TYP_DESC','Latitude','Longitude']]


#renaming the columns to be more user friendly and match it's own definition

crime_df=crime_df.rename(columns={'BORO_NM':'Borough','OFNS_DESC':'Crime Committed','PREM_TYP_DESC':'Location of Crime'})

print(crime_df.shape)
crime_df.head()

(108058, 5)


,Borough,Crime Committed,Location of Crime,Latitude,Longitude
0,BROOKLYN,PETIT LARCENY,STREET,40.656991,-73.876574
1,BROOKLYN,RAPE,STREET,40.674583,-73.930222
2,BRONX,PETIT LARCENY,DRUG STORE,40.830443,-73.871349
3,BRONX,PETIT LARCENY,GROCERY/BODEGA,40.817878,-73.916957
4,QUEENS,ASSAULT 3 & RELATED OFFENSES,OTHER,40.752011,-73.935872


Now that we got the columns we need from the dataset. We need to filter the data to get the crimes the restaurant owner is worried about.

In [44]:
crime_df['Crime Committed'].unique()

array(['PETIT LARCENY', 'RAPE', 'ASSAULT 3 & RELATED OFFENSES',
       'HARRASSMENT 2', 'DANGEROUS WEAPONS', 'MISCELLANEOUS PENAL LAW',
       'MURDER & NON-NEGL. MANSLAUGHTER', 'ROBBERY', 'FORGERY',
       'FELONY ASSAULT', 'CRIMINAL MISCHIEF & RELATED OF', 'SEX CRIMES',
       'GRAND LARCENY', 'OFFENSES AGAINST PUBLIC ADMINI', 'BURGLARY',
       'DANGEROUS DRUGS', 'FRAUDS', 'OFF. AGNST PUB ORD SENSBLTY &',
       'THEFT-FRAUD', 'UNAUTHORIZED USE OF A VEHICLE',
       'GRAND LARCENY OF MOTOR VEHICLE', 'VEHICLE AND TRAFFIC LAWS',
       'NYS LAWS-UNCLASSIFIED FELONY', "BURGLAR'S TOOLS",
       'CRIMINAL TRESPASS', 'INTOXICATED & IMPAIRED DRIVING', 'ARSON',
       'ALCOHOLIC BEVERAGE CONTROL LAW', 'OFFENSES AGAINST THE PERSON',
       'POSSESSION OF STOLEN PROPERTY', 'OTHER OFFENSES RELATED TO THEF',
       'ADMINISTRATIVE CODE', 'PROSTITUTION & RELATED OFFENSES',
       'OTHER STATE LAWS (NON PENAL LA', 'OFFENSES INVOLVING FRAUD',
       'PETIT LARCENY OF MOTOR VEHICLE', 'THEFT OF SERV

**Crime List:**

I created a crime list with the crimes that would affect the owner's restaurant. The crimes in the list ranges from vandalism to robbery. Then used the crime list to filter those crimes into the crime_df variable.


In [45]:
crime_list = ['ROBBERY','BURGLARY','CRIMINAL MISCHIEF & RELATED OF','GRAND LARCENY',"BURGLAR'S TOOLS",'CRIMINAL TRESPASS'
             ,'ARSON','OTHER OFFENSES RELATED TO THEF','THEFT OF SERVICES',]
crime_df = crime_df.loc[crime_df['Crime Committed'].isin(crime_list),:] #Filtering data to get the crimes in the list

In [46]:
print(crime_df.shape)
crime_df.head()

(29583, 5)


,Borough,Crime Committed,Location of Crime,Latitude,Longitude
14,BRONX,ROBBERY,RESIDENCE - APT. HOUSE,40.814771,-73.925111
21,MANHATTAN,CRIMINAL MISCHIEF & RELATED OF,STREET,40.800621,-73.942287
22,BROOKLYN,ROBBERY,RESIDENCE - APT. HOUSE,40.647950,-73.998409
25,BRONX,ROBBERY,STREET,40.814140,-73.903804
27,BRONX,CRIMINAL MISCHIEF & RELATED OF,STREET,40.856354,-73.850097


Filtering the location of crime to specifically "*restaurant/diner*" since the business is a restaurant.

In [47]:
pd.set_option('display.max_rows', 100) #Change the option to see all the rows in the series


#Seeing how many crimes were committed in each location 
crime_df['Location of Crime'].value_counts().sort_values(ascending=False)

STREET                          9600
RESIDENCE - APT. HOUSE          5165
RESIDENCE-HOUSE                 2603
RESIDENCE - PUBLIC HOUSING      1544
TRANSIT - NYC SUBWAY            1158
COMMERCIAL BUILDING             1046
OTHER                            766
CHAIN STORE                      755
RESTAURANT/DINER                 685
DEPARTMENT STORE                 487
GROCERY/BODEGA                   459
BAR/NIGHT CLUB                   381
CLOTHING/BOUTIQUE                310
MAILBOX OUTSIDE                  276
FAST FOOD                        232
PARK/PLAYGROUND                  222
DRUG STORE                       221
BANK                             207
PARKING LOT/GARAGE (PUBLIC)      199
HOTEL/MOTEL                      196
PARKING LOT/GARAGE (PRIVATE)     195
CONSTRUCTION SITE                193
GYM/FITNESS FACILITY             181
SMALL MERCHANT                   144
FOOD SUPERMARKET                 121
BEAUTY & NAIL SALON              114
PUBLIC SCHOOL                    113
T

In [48]:

rest_df = crime_df[crime_df['Location of Crime'] == 'RESTAURANT/DINER'] #Filtering dataframe to restaurants only


#Checking to make sure the number of rows match and the dataframe looks good
print(rest_df.shape)

rest_df.reset_index(inplace=True, drop=True) # Resetting the index 

rest_df.head()

(685, 5)


,Borough,Crime Committed,Location of Crime,Latitude,Longitude
0,MANHATTAN,GRAND LARCENY,RESTAURANT/DINER,40.800106,-73.965960
1,MANHATTAN,BURGLARY,RESTAURANT/DINER,40.855242,-73.929407
2,MANHATTAN,BURGLARY,RESTAURANT/DINER,40.759307,-73.962201
3,MANHATTAN,GRAND LARCENY,RESTAURANT/DINER,40.716585,-73.988761
4,QUEENS,BURGLARY,RESTAURANT/DINER,40.755876,-73.912392


Making sure there is no missing values in the "*rest_df*" dataframe

In [49]:
#Checking the percentage of the rows in each columns are null
rest_df.isnull().sum() / rest_df.shape[0]

Borough              0.0
Crime Committed      0.0
Location of Crime    0.0
Latitude             0.0
Longitude            0.0
dtype: float64

In [50]:
# Turning dataset into a csv to perform EDA on it
rest_df.to_csv('restaurant_crimes.csv',index=False)

,Borough,Crime Committed,Location of Crime,Latitude,Longitude
0,MANHATTAN,GRAND LARCENY,RESTAURANT/DINER,40.800106,-73.965960
1,MANHATTAN,BURGLARY,RESTAURANT/DINER,40.855242,-73.929407
2,MANHATTAN,BURGLARY,RESTAURANT/DINER,40.759307,-73.962201
3,MANHATTAN,GRAND LARCENY,RESTAURANT/DINER,40.716585,-73.988761
4,QUEENS,BURGLARY,RESTAURANT/DINER,40.755876,-73.912392
